In [ ]:
import sqlite3

conn = sqlite3.connect('../data_storage/spotify.db')

sql_query = '''
    SELECT playlist_track.pid, artist.artist_uri, artist.artist_name
    FROM playlist_track
    JOIN track ON playlist_track.track_uri = track.track_uri
    JOIN album ON track.album_uri = album.album_uri
    JOIN artist ON album.artist_uri = artist.artist_uri
'''


cursor = conn.execute(sql_query)

playlist_artists = {}

for row in cursor:
    playlist_id = row[0]
    artist_uri = row[1]
    artist_name = row[2]
    
    if playlist_id not in playlist_artists:
        playlist_artists[playlist_id] = []
    playlist_artists[playlist_id].append((artist_uri, artist_name))

conn.close()


In [ ]:
for playlist_id, artists in list(playlist_artists.items())[:10]:
    print(f"Playlist ID: {playlist_id}")
    for artist in artists:
        print(artist)
    

In [ ]:
artist_dict = {}

for playlist_id, artists in list(playlist_artists.items()):
    # print(f"Playlist ID: {playlist_id}")
    for artist in artists:
        uri = artist[0]
        name = artist[1]
        artist_dict[uri] = name 


Word2Vec

In [ ]:
from gensim.models import Word2Vec

vector_size = 200
window = 5
min_count = 1
sg = 1  

# Extract artist URIs from playlist_artists.values()
artist_uris = [[artist_uri for artist_uri, _ in artists] for artists in playlist_artists.values()]

# Create Word2Vec model using only artist URIs
model = Word2Vec(sentences=artist_uris, vector_size=vector_size, window=window, min_count=min_count, sg=sg)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np

# Assuming model.wv and artist_dict are already defined as in your previous snippet
embeddings = [model.wv[artist_key] for artist_key in model.wv.index_to_key]
embedding_array = np.array(embeddings)
tsne = TSNE(n_components=2, random_state=42)
embedded_artists = tsne.fit_transform(embedding_array)

# Plot the t-SNE visualization
plt.figure(figsize=(200, 200))
plt.scatter(embedded_artists[:, 0], embedded_artists[:, 1], alpha=0.5)

# Label points with artist names, escaping special characters
for i, artist_uri in enumerate(model.wv.index_to_key):
    artist_name = artist_dict.get(artist_uri, "Unknown Artist").replace("$", "\$")
    plt.text(embedded_artists[i, 0], embedded_artists[i, 1], artist_name, fontsize=10)

plt.title('t-SNE Visualization of Artist Embeddings')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()





As an alternative could use autoencoders 